# Quandl Tables API

In [1]:
import os
import datetime
import quandl

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl



In [2]:
def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(os.environ["HOME"], "quandl_data_table_downloads")
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink
    
    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    if date_override is None:
        my_date = datetime.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, filename=data_file, api_key=BrianBoonstraPrivateKeys['Quandl'], **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"


def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )




In [3]:
fetch_quandl_table('QUOTEMEDIA/TICKERS', avoid_download=False).head()

Data file f/Users/brian/quandl_data_table_downloads/QUOTEMEDIA/TICKERS_20220110.zip size 283962 exists already, no need to download


,ticker,exchange,company_name
0,BIOA_WS,NYSE,BioAmber Inc. Warrant expiring May 9 2017
1,CDE_WS,NYSE,Coeur D'Alene Mines Corporation
2,FINQ,NASDAQ,Purefunds Solactive FinTech ETF
3,IMED,NASDAQ,PureFunds ETFx HealthTech ETF
4,CVETV,NASDAQ,Covetrus Inc.


In [4]:
zfr = fetch_quandl_table('ZACKS/FR', avoid_download=False)
zfr.shape

Data file f/Users/brian/quandl_data_table_downloads/ZACKS/FR_20220110.zip size 33938302 exists already, no need to download


/Volumes/b2tb/conda_envs/2021-Apr-33150/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(574203, 40)

In [5]:
zfr.iloc[[0,1,2,-3,-2,-1],:8]

,m_ticker,ticker,comp_name,comp_name_2,exchange,currency_code,per_end_date,per_type
0,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2006-12-31,A
1,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2007-12-31,A
2,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2008-12-31,A
574200,BHN#,ZZLL,NaN,ZZLL INFO TEC,NaN,USD,2021-03-31,Q
574201,BHN#,ZZLL,NaN,ZZLL INFO TEC,NaN,USD,2021-06-30,Q
574202,BHN#,ZZLL,NaN,ZZLL INFO TEC,NaN,USD,2021-09-30,Q


In [6]:
prices = fetch_quandl_table('QUOTEMEDIA/PRICES')
prices.shape

Skipping any possible download of QUOTEMEDIA/PRICES


(41031912, 14)

In [7]:
prices.iloc[[0,1,2,1000000, 1000001,19000001,19000002,-3,-2,-1],:]

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,NJR,1991-07-23,18.50,18.63,18.38,18.63,11778.0,0.0,1.0,1.167836,1.176043,1.160261,1.176043,53001.000
1,NJR,1991-07-22,18.63,18.63,18.38,18.38,9200.0,0.0,1.0,1.176043,1.176043,1.160261,1.160261,41400.000
2,NJR,1991-07-19,18.63,18.75,18.50,18.63,9778.0,0.0,1.0,1.176043,1.183618,1.167836,1.176043,44001.000
1000000,DCO,1980-01-28,21.62,22.38,21.62,21.62,1186.0,0.0,1.0,5.205460,5.388446,5.205460,5.205460,4620.063
1000001,DCO,1980-01-29,21.62,21.62,20.87,20.87,2560.0,0.0,1.0,5.205460,5.205460,5.024882,5.024882,9972.480
19000001,LW,2017-04-20,42.31,42.74,42.02,42.60,415618.0,0.0,1.0,39.743467,40.147383,39.471058,40.015875,415618.000
19000002,LW,2017-04-19,42.29,42.39,42.01,42.20,526314.0,0.0,1.0,39.724680,39.818614,39.461665,39.640139,526314.000
41031909,NJR,1991-07-26,18.50,18.63,18.38,18.50,14978.0,0.0,1.0,1.167836,1.176043,1.160261,1.167836,67401.000
41031910,NJR,1991-07-25,18.25,18.50,18.13,18.38,9778.0,0.0,1.0,1.152055,1.167836,1.144479,1.160261,44001.000
41031911,NJR,1991-07-24,18.50,18.50,18.25,18.25,10000.0,0.0,1.0,1.167836,1.167836,1.152055,1.152055,45000.000
